In [ ]:
# pip install -U git+https://github.com/PrithivirajDamodaran/Gramformer.git
# pip install gramformer
# pip install torch
# pip install -U "spacy<3"
# ! -m spacy download en

In [7]:
from gramformer import Gramformer
import torch
import spacy

### Correcter

In [63]:
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(1212)

In [64]:
gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector


[Gramformer] Grammar error correct/highlight model loaded..


In [65]:
influent_sentences = [
    "He are moving here. We is happy.",
    "I am doing fine. How is you?",
    "How is they?",
    "Matt like fish",
    "the collection of letters was original used by the ancient Romans",
    "We enjoy horror movies",
    "Anna and Mike are going skiing?",
    "I walked to the store and I bought milk."
]   

for influent_sentence in influent_sentences:
    corrected_sentences = gf.correct(influent_sentence, max_candidates=1)
    print("[Input] ", influent_sentence)
    for corrected_sentence in corrected_sentences:
        print("[Correction] ",corrected_sentence)
    print("-" *100)

[Input]  He are moving here. We is happy.
[Correction]  He is moving here and we are happy.
----------------------------------------------------------------------------------------------------
[Input]  I am doing fine. How is you?
[Correction]  I am doing fine. How are you?
----------------------------------------------------------------------------------------------------
[Input]  How is they?
[Correction]  What are they?
----------------------------------------------------------------------------------------------------
[Input]  Matt like fish
[Correction]  Matt likes fish
----------------------------------------------------------------------------------------------------
[Input]  the collection of letters was original used by the ancient Romans
[Correction]  the collection of letters was originally used by the ancient Romans
----------------------------------------------------------------------------------------------------
[Input]  We enjoy horror movies
[Correction]  We enjoy horr

### Get Edits

In [ ]:
set_seed(1212)

gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector

influent_sentences = [
    "He are moving here.",
    "the collection of letters was original used by the ancient Romans",
    "We enjoys horror movies",
    "Anna and Mike is going skiing",
    "I will eat fish for dinner and drank milk",
    "what be the reason for everyone leave the comapny"
]   

for influent_sentence in influent_sentences:
    corrected_sentences = gf.correct(influent_sentence, max_candidates=1)
    print("[Input] ", influent_sentence)
    for corrected_sentence in corrected_sentences:
      print("[Edits] ", gf.get_edits(influent_sentence, corrected_sentence))
    print("-" *100)

[Gramformer] Grammar error correct/highlight model loaded..
[Input]  He are moving here.
[Edits]  [('VERB:SVA', 'are', 1, 2, 'is', 1, 2)]
----------------------------------------------------------------------------------------------------
[Input]  the collection of letters was original used by the ancient Romans
[Edits]  [('MORPH', 'original', 5, 6, 'originally', 5, 6)]
----------------------------------------------------------------------------------------------------
[Input]  We enjoys horror movies
[Edits]  [('VERB:SVA', 'enjoys', 1, 2, 'enjoy', 1, 2), ('NOUN', 'movies', 3, 4, 'movies.', 3, 4)]
----------------------------------------------------------------------------------------------------
[Input]  Anna and Mike is going skiing
[Edits]  [('VERB:SVA', 'is', 3, 4, 'are', 3, 4), ('OTHER', 'skiing', 5, 6, 'skiing.', 5, 6)]
----------------------------------------------------------------------------------------------------
[Input]  I will eat fish for dinner and drank milk
[Edits]  [

### Customize to our needs

In [59]:
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(1212)


gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector

def grammar_correction(user_input):

    corrected_sentence = gf.correct(user_input, max_candidates=1)
    corrected_sentence = list(corrected_sentence)[0] # make set into string
    print(type(corrected_sentence))

    if corrected_sentence != user_input:
        correction_message = f"[Correction] {corrected_sentence}"
        print(correction_message)

    return corrected_sentence, correction_message
    

[Gramformer] Grammar error correct/highlight model loaded..


In [60]:
ex1 = "I is to the store. I buy chocolate. I see u tomorrow"

In [61]:
correction = grammar_correction(ex1)
correction

<class 'str'>
[Correction] I went to the store. I bought chocolate. I see u tomorrow.


('I went to the store. I bought chocolate. I see u tomorrow.',
 '[Correction] I went to the store. I bought chocolate. I see u tomorrow.')

### Class

In [88]:
class GrammarModel(Gramformer):
    """
    """
    def __init__(self, models=1, use_gpu=False, seed=1212):
        self.gf = super().__init__(models=1, use_gpu=False)
        self.seed = seed
        set_seed(self.seed)


    def set_seed(self, seed):
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)


    def grammar_correction(self, user_input):
        """
        Generate a corrected sentence and a message to the user with the correction.
        """

        corrected_sentence = self.correct(user_input, max_candidates=1)
        corrected_sentence = list(corrected_sentence)[0]

        if corrected_sentence != user_input:
            correction_message = f"[Correction] {corrected_sentence}"
            print(correction_message)

        return corrected_sentence, correction_message


    def add_correction_to_chat_history(self, chat_history):
        """
        Append the message to the user to the chat history.
        Return the corrected sentence ( to be used as input )
        """
        pass

    def _get_edits(self, user_input, corrected_sentence):
        user_input = self.annotator.parse(user_input)
        corrected_sentence = self.annotator.parse(corrected_sentence)
        alignment = self.annotator.align(user_input, corrected_sentence)
        edits = self.annotator.merge(alignment)

        if len(edits) == 0:  
            return []

        edit_annotations = []
        for e in edits:
            e = self.annotator.classify(e)
            edit_annotations.append(e.type[2:])
                
        if len(edit_annotations) > 0:
            return edit_annotations
        else:    
            return []


In [89]:
gm = GrammarModel(models = 1, use_gpu=False) # 1=corrector, 2=detector


[Gramformer] Grammar error correct/highlight model loaded..


In [90]:
correct_sentence, message = gm.grammar_correction(ex1)

[Correction] I am in the store. I buy chocolate. I see you tomorrow.


In [91]:
error_types = gm.get_edits(ex1, correct_sentence)
error_types

['VERB:SVA', 'PREP', 'OTHER', 'NOUN']